# Masques d’attention
Les masques d’attention sont des tenseurs ayant exactement la même forme que le tenseur d’identifiants d’entrée, remplis de 0 et de 1 :

1 indique que les tokens correspondants doivent être analysés
0 indique que les tokens correspondants ne doivent pas être analysés (c’est-à-dire qu’ils doivent être ignorés par les couches d’attention du modèle).
Complétons l’exemple précédent avec un masque d’attention :

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:

batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

In [3]:
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0], # 0 pour ignorer pad_token_id
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids),attention_mask=torch.tensor(attention_mask)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


Nous avons maintenant obtenu le même résultat, que ce soit en passant des phrases individuelles de différentes longueurs dans notre modèle ou un lot de phrases identiques avec du padding, après avoir configuré les couches d’attention pour ignorer les tokens de padding.

Remarquez comment la dernière valeur de la deuxième séquence est un identifiant de padding valant 0 dans le masque d’attention.

# Séquences plus longues
Les transformers acceptent en entrée que des séquences d’une longueur limitée. La plupart des modèles traitent des séquences allant jusqu’à 512 ou 1024 tokens et plantent lorsqu’on leur demande de traiter des séquences plus longues. Il existe deux solutions à ce problème :

utiliser un modèle avec une longueur de séquence supportée plus longue,
tronquer les séquences.
Certains modèles sont spécialisés dans le traitement de très longues séquences comme par exemple le Longformer ou le LED. Si vous travaillez sur une tâche qui nécessite de très longues séquences, nous vous recommandons de jeter un coup d’œil à ces modèles.

Sinon, nous vous recommandons de tronquer vos séquences en spécifiant le paramètre max_sequence_length :


sequence = sequence[:max_sequence_length]


# Tout assembler

Dans les dernières sections, nous avons fait de notre mieux pour effectuer la plupart du travail manuellement. Nous avons exploré le fonctionnement des tokenizers et examiné la tokenisation, la conversion en identifiants d’entrée, le padding, la troncature et les masques d’attention.

Cependant, comme nous l’avons vu dans la section 2, l’API 🤗 Transformers peut gérer tout cela pour nous via une fonction dans laquelle nous allons nous plonger ici. Lorsque vous appelez votre tokenizer directement sur la phrase, vous récupérez des entrées qui sont prêtes à être passées dans votre modèle :

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."
# J'ai attendu un cours d’HuggingFace toute ma vie.

model_inputs = tokenizer(sequence)

Ici, la variable model_inputs contient tout ce qui est nécessaire au bon fonctionnement d’un modèle. Pour DistilBERT, cela inclut les identifiants d’entrée ainsi que le masque d’attention. D’autres modèles qui acceptent des entrées supplémentaires sont également fournis par l’objet tokenize

Comme nous allons le voir dans les quelques exemples ci-dessous, cette méthode est très puissante. Premièrement, elle peut tokeniser une seule séquence :

In [8]:
sequence = "I've been waiting for a HuggingFace course my whole life."
# J'ai attendu un cours d’HuggingFace toute ma vie.


model_inputs = tokenizer(sequence)


Elle gère également plusieurs séquences à la fois, sans modification de l’API :



In [9]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!",
]  # « J'ai attendu un cours de HuggingFace toute ma vie. », « Moi aussi ! »

model_inputs = tokenizer(sequences)

Il est possible de faire du padding selon plusieurs objectifs :



In [10]:
# Remplit les séquences jusqu'à la longueur maximale de la séquence
model_inputs = tokenizer(sequences, padding="longest")

# Remplit les séquences jusqu'à la longueur maximale du modèle (512 pour BERT ou DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Remplit les séquences jusqu'à la longueur maximale spécifiée
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

La fonction peut également tronquer les séquences :



In [11]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!",
]  # « J'ai attendu un cours de HuggingFace toute ma vie. », « Moi aussi ! »

# Tronque les séquences qui sont plus longues que la longueur maximale du modèle
# (512 pour BERT ou DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Tronque les séquences qui sont plus longues que la longueur maximale spécifiée
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

L’objet tokenizer peut gérer la conversion en des tenseurs de frameworks spécifiques. Ils peuvent ensuite être directement envoyés au modèle. Par exemple, dans le code suivant, nous demandons au tokenizer de retourner des tenseurs PyTorch lorsque l’on spécifie "pt", de retourner des tenseurs TensorFlow lorsque l’on spécifie "tf" et des tableaux NumPy lorsque l’on indique "np" :

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!",
]  # « J'ai attendu un cours de HuggingFace toute ma vie. », « Moi aussi ! »

# Retourne des tenseurs PyTorch
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Retourne des tenseurs TensorFlow
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Retourne des tableaux NumPy
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

Si nous jetons un coup d’œil aux identifiants d’entrée renvoyés par le tokenizer, nous verrons qu’ils sont un peu différents de ceux que nous avions précédemment :

In [12]:
sequence = "I've been waiting for a HuggingFace course my whole life."
# « J'ai attendu un cours de HuggingFace toute ma vie. »

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


Un identifiant symbolique a été ajouté au début ainsi qu’un autre à la fin. Décodons les deux séquences d’identifiants ci-dessus pour voir de quoi il s’agit

In [13]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


Le tokenizer a ajouté le mot spécial [CLS] au début et le mot spécial [SEP] à la fin. C’est parce que le modèle a été pré-entraîné avec ces mots, donc pour obtenir les mêmes résultats pour l’inférence, nous devons également les ajouter. Notez que certains modèles n’ajoutent pas de mots spéciaux, ou en ajoutent des différents. Les modèles peuvent aussi ajouter ces mots spéciaux seulement au début, ou seulement à la fin. Dans tous les cas, le tokenizer sait lesquels sont attendus et s’en occupe pour vous

# Conclusion : du <i> tokenizer </i> au modèle
Maintenant que nous avons vu toutes les étapes individuelles que l’objet tokenizer utilise lorsqu’il est appliqué sur des textes, voyons une dernière fois comment il peut gérer plusieurs séquences (padding), de très longues séquences (troncation) et plusieurs types de tenseurs avec son API principale :

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!",
]  # « J'ai attendu un cours de HuggingFace toute ma vie. », « Moi aussi ! »


tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
